<h1>Learning Blackjack with Q-Learning</h1>

<img src="files/cards.jpg">

This notebook explores the game of Blackjack, and attempts by an agent to learn an optimal strategy to win (or minimize losses) over several hands. This was created with the goal to learn more about object oriented programing in python and Q-learning rather than an actual attempt to "beat" the game. A simplified version of the rules was used:

<h3>The Rules</h3>

1. The player is dealt 2 cards from an infinite deck. Since the deck is infinite, knowledge about the player's cards does not provide any knowledge about the dealer's cards or about the card that will be drawn next.

2. The player then has the option to "hit" or "stand".  If the player chooses to hit, he is dealt another card.

3. Step 2 is repeated until either the player chooses to stand, or the players hand value is greater than 21.

4. If the players hand value is greater than 21, then he busts, the dealer wins, and the hand is over.

5. If the players hand value is less than or equal to 21 then the dealer must make a turn.

6. During the dealers hand, the dealer must continue to hit while his hand value is less than 17.  When the dealer's hand value is 17 or over, the dealer stands and the hand is over.

7. If the dealer's hand value is over 21, the dealer busts, the player wins, and the hand is over.

8. If neither player went bust during his turn, then whoever has the higher hand value wins. If the player and the dealer have the same hand value then the game is a draw.

<h3>Code</h3>

The code to implement the rules and train the agent are located in https://github.com/charlieaxle/blackjack-qlearn/blob/master/blackjack.py.  

In [1]:
import blackjack

<h5>RandomCard</h5>

In [2]:
help(blackjack.Hand)

Help on class Hand in module blackjack:

class Hand(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, initial_cards=0)
 |      Initializes a new Hand object
 |      
 |      Attributes:
 |          cards (list): A list that may contain multiple Card objects
 |          score (int): The current value of the cards in the hand
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  deal_card(self, num_cards=1)
 |  
 |  get_score(self)
 |      Used to access the current score of a Hand object
 |      Params: None
 |      Returns: self.score
 |  
 |  is_bust(self)
 |  
 |  possible_moves(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



<h5>Hand</h5>

In [3]:
print(blackjack.Hand.__doc__)


    
    An object that represents multiple cards
    
    Attributes:
        cards (list): A list that may contain multiple Card objects
        score (int): The current value of the cards in the hand
        
    Methods:
        get_score(): Getter function to return score of hand
        deal_card(num_cards = 1): Adds one or more Cards to hand
            Params:
                num_cards (int): Amount of cards to be added to hand
        is_bust(): Returns True if hand is bust, otherwise returns false
        possible_moves(): Returns an array of possible moves available
    
    
